In [ ]:
import pandas as pd
from google.colab import files

# Passo 1: Fazer upload do arquivo da planilha
print("Por favor, envie o arquivo da planilha:")
uploaded = files.upload()

# Identificar o nome do arquivo enviado
nome_arquivo = list(uploaded.keys())[0]


# Exibir os nomes das colunas para confirmar quais estão disponíveis
print()

# Ler a planilha (formato Excel ou CSV)
if nome_arquivo.endswith(".xlsx"):
    df = pd.read_excel(nome_arquivo)
elif nome_arquivo.endswith(".csv"):
    df = pd.read_csv(nome_arquivo)
else:
    raise ValueError("O arquivo precisa ser no formato Excel (.xlsx) ou CSV (.csv)")

# Passo 2: Definir categorias e criar estrutura para a matriz
categorias = ["Aparência", "Caráter", "Papel Social", "Emoção", "Outro"]
matriz = pd.DataFrame(columns=["Item"] + categorias)

# Normalizar as strings das categorias para evitar problemas de contagem
df.replace({cat: cat.strip() for cat in categorias}, inplace=True)

# Passo 3: Contar ocorrências por categoria para cada item
for idx, row in df.iterrows():
    # Selecionar as colunas de classificação (ajuste os nomes conforme necessário)
    classificacoes = row[["anotadora 1", "anotadora 2", "anotadora 3", "anotadora 4", "anotadora 5", "anotadora 6", "anotadora 7"]]

    # Normalizar as strings antes da contagem
    classificacoes = classificacoes.str.strip()

    # Contar quantas vezes cada categoria aparece
    contagem = {cat: (classificacoes == cat).sum() for cat in categorias}
    contagem["Item"] = row["id_item"]  # Usar a coluna identificadora correta

    # Adicionar a linha à matriz
    matriz = pd.concat([matriz, pd.DataFrame([contagem])], ignore_index=True)


# Exibir a matriz gerada
print("Matriz gerada com sucesso!")
print(matriz)




In [ ]:

from statsmodels.stats.inter_rater import fleiss_kappa

contagens = matriz.drop(columns=["Item"])

contagens = contagens.astype(int)

kappa = fleiss_kappa(contagens.to_numpy(), method='fleiss')

print(f"Fleiss' Kappa: {kappa:.4f}")




Fleiss' Kappa: 0.3395


In [ ]:
# Número esperado de avaliadores
num_avaliadores = 7

# Soma das classificações por item
somas_por_linha = contagens.sum(axis=1)

# Identificar linhas com soma diferente do número esperado de avaliadores
linhas_problema = somas_por_linha != num_avaliadores

# Exibir as linhas problemáticas
if linhas_problema.any():
    print("Linhas com problemas encontradas:")
    print(matriz.loc[linhas_problema])  # Inclui os IDs e contagens problemáticas
else:
    print("Nenhuma linha problemática encontrada.")


Nenhuma linha problemática encontrada.
